In [1]:
### EXP2.2: EN -> DE

In [2]:
### Install Required Packages
!pip install jsonlines sentencepiece # nltk evaluate unbabel-comet accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.3 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
drive_PATH = '../content/drive/MyDrive/Colab Notebooks/l101.experiments.1'

Mounted at /content/drive


In [4]:
# General
import numpy as np
import tqdm
import random

# Data
import jsonlines
import torch

# Model
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M") # Load Model
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M") # Load Tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): Embedding(128112, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): Embedding(128112, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-11): 12 x M2M100EncoderLayer(
          (self_attn): M2M100Attention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm

In [5]:
### Dataset
test_tr_PATH = drive_PATH + '/data/datasets/test_tr_1000.jsonl'
test_de_PATH = drive_PATH + '/data/datasets/test_de_1000.jsonl'
test_tr = []
test_de = []

# with jsonlines.open(test_tr_PATH) as f:
#     for line in f.iter():
#         test_tr.append(line)
with jsonlines.open(test_de_PATH) as f:
    for line in f.iter():
        test_de.append(line)

In [6]:
# SRC: EN
# TGT: DE
src_lang = "en"
tgt_lang = "de"

In [7]:
### Setting the Language Configurations
dataset = test_de
tokenizer.src_lang = src_lang
print(len(dataset))

1000


In [11]:
### Decoding: Epsilon Sampling

res = []
temperature = 1
epsilon_cutoff = 0.02
num_candidates = 128

for sent in tqdm.tqdm(dataset):
  #print(sent['en'])
  src_sent = sent[src_lang]
  encoded_src_sent = tokenizer(src_sent, return_tensors="pt").input_ids.to("cuda")

  candidates = []
  for idxc in range(num_candidates):
    tgt_tokens = model.generate(encoded_src_sent, forced_bos_token_id=tokenizer.get_lang_id(tgt_lang), do_sample=True, temperature=temperature, epsilon_cutoff=0.02, num_beams=1, early_stopping=False, num_return_sequences=1)
    tgt_sent = tokenizer.batch_decode(tgt_tokens, skip_special_tokens=True)
    candidates.append(tgt_sent)

  res_sent = {'src': sent[src_lang], 'candidates': candidates, 'ref':sent[tgt_lang]}
  res.append(res_sent)

cand_de_PATH = drive_PATH+ f'/exp2.2.ende-n{num_candidates}-t{temperature}-e{epsilon_cutoff}.jsonl'
with jsonlines.open(cand_de_PATH, mode='w') as writer:
    for item in res:
        writer.write(item)

100%|██████████| 1000/1000 [17:21:55<00:00, 62.52s/it]


In [10]:
res[0]

{'src': 'However, the district is lacking additional personnel which it needs to overcome growing demands and duties.',
 'candidates': [['Doch die Region fehlt dem zusätzlichen Personal, dem sie nötig ist, um die wachsenden Ansprüche und Verpflichtungen zu überwinden.'],
  ['Der Distrikt ist jedoch von zusätzlichen Mitarbeitern fehl, die er brauchen, um steigende Anforderungen und Pflichten zu überwinden.'],
  ['Doch die Umgebung fehlt zusätzlich Personalität, die es benötigt, um wachsende Forderungen und Schuld zu überwinden.'],
  ['Doch die Bezirke fehlt mehr Personal, den sie benötigt, um die wachsenden Anforderungen und Pflichten zu überwinden.'],
  ['Der Bezirk fehlt allerdings zusätzliches Personal, das er benötigt, um zunehmend anforderungen und Pflichten zu überwinden.'],
  ['Allerdings fehlte der Bezirk zusätzliche Mitarbeiter, die es braucht, um die wachsenden Bedürfnisse und Pflichten zu überwinden.'],
  ['Doch im Bezirk fehlt zusätzlich das Personal, das es braucht, um den 